# Content
Files prepared: 
* stock_list.xlsx = 3800 stock names + exchange names
* names_all.xlsx = 3208 stock names + Indicies


## 1) Installing Packages

In [ ]:
# 1.) General
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure  
import numpy as np
import math
from pandas import DataFrame
from math import sqrt
from datetime import datetime
import csv
import time
from datetime import date

# 2.) Scrapping
import requests                  
import lxml.html as lh
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains      # ?????
from time import sleep

import shutil

## 2) Loading list of 3800 stock names and exchanges

In [ ]:
path = "C:/Users/Desktop/Python/stocks/"

names = pd.read_excel(path + "stock_list.xlsx" )
names.head()

## 3) Scraping Indicies from Yahoo Finance
### 4-5 hours !!

In [ ]:
import time

def get_indicies(stock_names):
    
    names = []
     
    # open browser    
    browser = webdriver.Chrome()
    url = "https://finance.yahoo.com"
    browser.get(url)
    
    # click on AGREE - might be changed time by time
    agree = browser.find_elements_by_xpath("/html/body/div/div/div/div/div[2]/div[2]/form/button")[0]
    hover = ActionChains(browser).move_to_element(agree)
    hover.perform()
    agree.click()
    
    # The whole loop is processed in a single browser window
    for i in range(len(stock_names)):      
        
        # Click to the search row
        find_stock = browser.find_elements_by_xpath('/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]')[0]
        hover = ActionChains(browser).move_to_element(find_stock)
        hover.perform()
        find_stock.click()
        
        # Insert stock name
        find_stock.send_keys(stock_names.loc[i].Names)  
        ### ILOC works with the real index of the row (starts with 0), while LOC works with the original values, before slicing (until we reset index)
        ### in this case both can be used
            
        time.sleep(1)
        ### Helps a little, but still, sometimes the last searched value is automatically put in the search row... could be checked in the end
        
        # try click on Search button
        try: 
            find_button = browser.find_elements_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/div[1]/button")[0]
            hover = ActionChains(browser).move_to_element(find_button)
            hover.perform()
            find_button.click()

        # Wait a sec; If there is a name, take it
            time.sleep(1)
            name = browser.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[4]/div/div/div/div[2]/div[1]/div[1]/h1").text
            names.append(name)
            
        # If failed, save error and open the browser, again
        except:
            names.append("error")
            url = "https://finance.yahoo.com"
            browser.get(url)
    
            # click on AGREE, if it shows up
            try:
                agree = browser.find_elements_by_xpath("/html/body/div/div/div/div/div[2]/div[2]/form/button")[0]
                hover = ActionChains(browser).move_to_element(agree)
                hover.perform()
                agree.click()
            except:
                pass
        
        print(str(i))
        print(stock_names.loc[i].Names)
        time.sleep(1)
    
    # In the end, quit the browser
    browser.quit()
 
    return(names)   

In [ ]:
### Takes 4-5 hours !!

INDICIES = get_indicies(#names)
np.savetxt(path + "indicies.csv", INDICIES, delimiter=",", fmt='%s', encoding = "ISO-8859-1")

### 3.1 Check of Errors

In [ ]:
df = pd.Series([])
data = pd.read_csv(path + "indicies.csv", header = None, sep = ":", encoding = "ISO-8859-1")
df = df.append(data[0])

display(len(df))
df.reset_index(drop= True, inplace = True)

# Percentage of errors
display(len(df[df == "error"])/len(df))
df.head()

### 3.2 Adding Indicies to Names

In [ ]:
names["Indicies"] = df.copy() 
names.head()

### 3.3 Getting indicies out of parentheses

In [ ]:
def indicies(s):

    x = s[s.find("(")+1:s.find(")")]
    return x

names["Quotes"] = names["Indicies"].apply(indicies)
names.tail()

### 3.4 Check for Duplicates, Drop

In [ ]:
display(len(names))

# Check duplicates
names[names["Quotes"].duplicated( keep = False)].sort_values(by = ["Quotes"], axis=0)

# Drop duplicates
names.drop_duplicates(subset ="Quotes",keep = "first", inplace = True) 
names.reset_index(drop= True, inplace = True)

display(len(names))

## 4) Scraping daily data for Indicies
### OLD VERSION, TOO SLOW  -> Try 5) !!

In [ ]:
def data_dwn_OLD(stocks):

    for i in stocks:
        
        # For each stock open browser - could be better ?
        browser = webdriver.Chrome()
                                                                 # This time period corresponds to all the history up to the 31.12.2019
        url = "https://finance.yahoo.com/quote/" + str(i) + "/history?period1=10104800&period2=1577919600&interval=1d&filter=history&frequency=1d"
                                
        browser.get(url)

        
        # This clicks on AGREE, if there is a warning     
        try:
            agree = browser.find_elements_by_xpath("/html/body/div/div/div/div/div[2]/div[2]/form/button")[0]
            hover = ActionChains(browser).move_to_element(agree)
            hover.perform()
            agree.click()

        except:
            pass
        
        sleep(2)
        
        # Clicks on Download -> CSV is downloaded
        play = browser.find_elements_by_xpath('/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[1]/div[2]/span[2]/a/span')[0]       
        hover = ActionChains(browser).move_to_element(play)
        hover.perform()
        play.click()
        
        sleep(2)
        
        browser.quit()
   

In [ ]:

data_dwn_OLD(names.Quotes)

## 5) Scraping daily data for Indicies
### Better version, YAHOO API instead of pressing download button. No Selenium needed
*  3208 -> 3067 scraped
* takes about 1 hour

### 5.1) Load the data

In [ ]:
names = pd.read_excel(path + "names_all.xlsx")
names.head()

In [ ]:
def data_dwn(stocks):
    
    # One web browser window is needed
    browser = webdriver.Chrome()
    
    # for all 3208 stocks, use API to download CSVs
    for i in stocks:
                                                                            # This time period corresponds to all the history up to the 31.12.2019
        url = "https://query1.finance.yahoo.com/v7/finance/download/" + str(i) + "?period1=10104800&period2=1577919600&interval=1d&events=history"
                                
        browser.get(url)
        sleep(0.5)
    browser.quit()
    

In [ ]:
# Takes about 1 hour

data_dwn(names.Quotes)

## 6.) Indicies -> Countries
* In the orginal dataset ???

In [ ]:
def country(x):
    y = x.split(".")
    if len(y) > 1:
        return y[-1]
    else:
        return "-"

names["Country"] = names["Quotes"].apply(country)
names.head()

### 6.1 List all countries

In [ ]:
countries = names_all.Country.unique()
display(len(countries))


In [ ]:
names.groupby('Country')['Quotes'].nunique()